In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = bassamsabber
%env KAGGLE_KEY = 717678f8ebfb53d84e9c47444c275871
!kaggle competitions download -c computationalintelligencesc2020
!unzip DataSet
!unzip computationalintelligencesc2020

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import cv2
from tqdm import tqdm 
import os
import pandas as pd
from os.path import join
from random import shuffle
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from keras.initializers import glorot_uniform
from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras.backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras import optimizers, layers
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Conv2D, SeparableConv2D, Add, Dense, BatchNormalization, ReLU, MaxPool2D, GlobalAvgPool2D, Concatenate, Average,Maximum
from keras.utils.data_utils import get_file
print("Done")

Done


In [ ]:
#seed
import numpy as np
import tensorflow as tf
import random as rn

import os
os.environ['PYTHONHASHSEED'] = '0'

np.random.seed(42)

rn.seed(12345)

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K


tf.compat.v1.set_random_seed(1234)

sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [ ]:
f_1 ="/content/Scenes training set/Scenes training set/buildings"
f_2 = "/content/Scenes training set/Scenes training set/forest"
f_3 = "/content/Scenes training set/Scenes training set/glacier"
f_4 = "/content/Scenes training set/Scenes training set/mountain"
f_5 = "/content/Scenes training set/Scenes training set/sea"
f_6 = "/content/Scenes training set/Scenes training set/street"


#Detection often requires fine-grained visual information 
    ####so we increase the input resolution of the network from 150x150 to 224x224
IMG_SIZE = 224

def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(f_1)):
        path = os.path.join(f_1, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([1,0,0,0,0,0])]) 
        
    for img in tqdm(os.listdir(f_2)):
        path = os.path.join(f_2, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,1,0,0,0,0])])
    
    for img in tqdm(os.listdir(f_3)):
        path = os.path.join(f_3, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,1,0,0,0])])

    for img in tqdm(os.listdir(f_4)):
        path = os.path.join(f_4, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,1,0,0])])

    for img in tqdm(os.listdir(f_5)):
        path = os.path.join(f_5, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,0,1,0])])
    for img in tqdm(os.listdir(f_6)):
        path = os.path.join(f_6, img) 
        img = cv2.imread(path) 
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        #out  = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        training_data.append([np.array(img), np.array([0,0,0,0,0,1])])

    shuffle(training_data)
    #train_data=np.load('/content/drive/My Drive/data/train_data.npy', allow_pickle=True) 
    np.save('train.npy', training_data)
    return training_data 


####train_dataa = create_train_data() 
training_data = create_train_data() 

100%|██████████| 2382/2382 [00:01<00:00, 1321.95it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
TEST_DIR = "/content/Scenes testing test/Scenes testing test"
def Create_test_data():
    testing_data = [] 
    for img in tqdm(os.listdir(TEST_DIR)):
        name=img
        path = os.path.join(TEST_DIR, img) 
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        testing_data.append([np.array(img),name])
    np.save('test.npy', testing_data) 
    return testing_data

test_data = Create_test_data()

100%|██████████| 3050/3050 [00:02<00:00, 1372.79it/s]
/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [ ]:
 train_data=np.load("/content/drive/MyDrive/natural scense/train.npy", allow_pickle=True) 
 test_data=np.load("/content/drive/MyDrive/natural scense/test.npy", allow_pickle=True) 

In [ ]:
#Load weights of VGG19 NO TOP ON IMAGENET
##To make transfer learning
##
####
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
weights_path = get_file('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',WEIGHTS_PATH_NO_TOP)

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
#padding: one of "valid" or "same" (case-insensitive). "valid" means no padding. 
###"same" results in padding evenly to the left/right or up/down of the input such that output has the same height/width dimension as the input.


In [ ]:
def VGG16(include_top=False,weights='imagenet',
          classes=6):
  
  input = Input(shape=(224, 224, 3))

  x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(input)
  x = Conv2D(filters=64, kernel_size=3, padding='same', activation='relu')(x)
  x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

  x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=128, kernel_size=3, padding='same', activation='relu')(x)
  x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

  x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=256, kernel_size=3, padding='same', activation='relu')(x)
  x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = Conv2D(filters=512, kernel_size=3, padding='same', activation='relu')(x)
  x = MaxPool2D(pool_size=2, strides=2, padding='same')(x)

  # x = Flatten()(x)
  # x = Dense(units=4096, activation='relu')(x)
  # # x = Dropout(0.2)(x)
  # x = Dense(units=4096, activation='relu')(x)
  # # x = Dropout(0.2)(x)
  # output = Dense(units=2, activation='softmax')(x)

  model = Model(inputs=input, outputs=x)

  model.load_weights(weights_path)

  return model

model = VGG16(include_top=False, weights='imagenet', classes=6)
new_model = model.output
#model.trainable = False
#Pooling layers:GlobalAveragePooling2D::Global average pooling operation for spatial data.
global_layer = GlobalAveragePooling2D() 
new_model = global_layer(new_model)
new_model = Dense(3072, activation="relu")(new_model)
new_model = Dense(6, activation="softmax")(new_model)


In [ ]:
vgg_model = Model(inputs=model.input, outputs=new_model)

In [ ]:
vgg_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0   

In [ ]:
IMG_SIZE = 224



#momentum: float hyperparameter >= 0 that accelerates gradient descent in the relevant direction and dampens oscillations.
#Defaults to 0, i.e., vanilla gradient descent.
#nesterov: boolean. Whether to apply Nesterov momentum. Defaults to False.



optimization = optimizers.SGD(lr=.0001,decay=1e-6 
                              ,momentum=0.9,nesterov=True,name="SGD")

vgg_model.compile(optimizer=optimization, loss='categorical_crossentropy', metrics=['accuracy'])

x_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 

y_train = np.asarray([i[1] for i in train_data])

vgg_model.fit(x_train, y_train, batch_size=8, epochs=10, verbose=1)
  
vgg_model.save("vgg16.model")

Epoch 1/10
1755/1755 [==============================] - 152s 69ms/step - loss: 0.3954 - accuracy: 0.8686
Epoch 2/10
1755/1755 [==============================] - 120s 68ms/step - loss: 0.1644 - accuracy: 0.9444
Epoch 3/10
1755/1755 [==============================] - 120s 68ms/step - loss: 0.1216 - accuracy: 0.9567
Epoch 4/10
1755/1755 [==============================] - 120s 68ms/step - loss: 0.0812 - accuracy: 0.9726
Epoch 5/10
1755/1755 [==============================] - 119s 68ms/step - loss: 0.0651 - accuracy: 0.9785
Epoch 6/10
1755/1755 [==============================] - 120s 68ms/step - loss: 0.0455 - accuracy: 0.9858
Epoch 7/10
1755/1755 [==============================] - 120s 69ms/step - loss: 0.0329 - accuracy: 0.9887
Epoch 8/10
1755/1755 [==============================] - 120s 69ms/step - loss: 0.0195 - accuracy: 0.9944
Epoch 9/10
1755/1755 [==============================] - 120s 68ms/step - loss: 0.0157 - accuracy: 0.9953
Epoch 10/10
1755/1755 [==============================] 

In [ ]:
Submit=[]
for num, data in enumerate(test_data[:]): 
    img_label = data[0]
    name = data[1] 
    data = img_label.reshape(-1,224, 224, 3) 
    model_out = vgg_model.predict([data])[0]
    if np.argmax(model_out) == 0:
        Submit.append([name,0])
    elif np.argmax(model_out) == 1:
        Submit.append([name,1])
    elif np.argmax(model_out) == 2:
        Submit.append([name,2])
    elif np.argmax(model_out) == 3:
        Submit.append([name,3])
    elif np.argmax(model_out) == 4:
        Submit.append([name,4])
    elif np.argmax(model_out) == 5:
        Submit.append([name,5])                                 
print(Submit)
Submied=pd.DataFrame(Submit,columns=['Image','Label'])
Submied.to_csv("/content/submit.csv", index = False) 

[['23004.jpg', 3], ['23215.jpg', 5], ['20998.jpg', 2], ['20627.jpg', 2], ['23885.jpg', 4], ['23291.jpg', 3], ['22374.jpg', 4], ['20683.jpg', 0], ['22636.jpg', 3], ['22623.jpg', 4], ['20647.jpg', 4], ['21277.jpg', 4], ['22450.jpg', 5], ['24115.jpg', 3], ['21967.jpg', 2], ['20283.jpg', 4], ['23597.jpg', 1], ['20751.jpg', 2], ['22311.jpg', 5], ['24213.jpg', 2], ['22346.jpg', 3], ['22400.jpg', 1], ['22490.jpg', 1], ['21936.jpg', 4], ['24175.jpg', 5], ['20334.jpg', 3], ['23024.jpg', 3], ['22149.jpg', 1], ['21017.jpg', 0], ['22268.jpg', 1], ['24132.jpg', 4], ['24250.jpg', 5], ['23429.jpg', 3], ['21230.jpg', 2], ['546.jpg', 0], ['23932.jpg', 0], ['22097.jpg', 5], ['23419.jpg', 4], ['22724.jpg', 3], ['20807.jpg', 5], ['22064.jpg', 5], ['21287.jpg', 5], ['20261.jpg', 1], ['949.jpg', 0], ['21186.jpg', 2], ['22895.jpg', 4], ['23177.jpg', 2], ['23876.jpg', 1], ['20726.jpg', 2], ['24133.jpg', 2], ['22614.jpg', 3], ['21294.jpg', 5], ['20096.jpg', 0], ['22158.jpg', 3], ['20435.jpg', 3], ['24297.jpg',

In [ ]:
#Ensemble is done because an ensemble tends to perform better than singles improving the generalization ability.

#CONCATENATION ENSEMBLE
#The concatenation is the most common technique to merge different data sources.
# A concatenation ensemble receives different inputs, whatever their dimensions are, and concatenates them on a given axis.
# A drawback, of attaching information side by side, can be the dimensionality explosion.

In [ ]:
model1 = tf.keras.models.load_model("/content/drive/MyDrive/natural scense/vgg19(true).model")
#freezing all the weights:before compiling the model
model1.trainable = False
for i, layer in enumerate(model1.layers):
  #Change layers names !
    layer._name = 'm1_layer_' + str(i)


In [ ]:
model2 = tf.keras.models.load_model("/content/drive/MyDrive/natural scense/vgg16.model")
model2.trainable = False
for i, layer in enumerate(model2.layers):
    layer._name = 'm2_layer_' + str(i)

In [ ]:
merge = Concatenate()([model1.output,model2.output])
merge = tf.keras.layers.Dense(3072, activation='relu')(merge)
output = tf.keras.layers.Dense(6, activation='softmax')(merge)
Final_model = tf.keras.models.Model(inputs=[model1.input,model2.input],
outputs=output)

In [ ]:
IMG_SIZE = 224
optimization = optimizers.SGD(lr=.001,decay=1e-6 ,momentum=0.9,nesterov=True,name="SGD")
Final_model.compile(optimizer=optimization, loss='categorical_crossentropy', metrics=['accuracy'])

x_train1= np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3) 
x_train2= np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)

y_train = np.asarray([i[1] for i in train_data])

#By setting verbose 0, 1 or 2 you just say how do you want to 'see' the training progress for each epoch.
Final_model.fit([x_train1,x_train2], y_train, batch_size=8, epochs=10, verbose=1)
  
Final_model.save("ensemble.model")

In [ ]:
Submit=[]
for num, data in enumerate(test_data[:]): 
    img_label = data[0]
    name=data[1] 
    data1 = img_label.reshape(-1,224, 224, 3) 
    data2 = img_label.reshape(-1,224, 224, 3)
    model_out = Final_model.predict([data1,data2])[0]
    if np.argmax(model_out) == 0:
        Submit.append([name,0])
    elif np.argmax(model_out) == 1:
        Submit.append([name,1])
    elif np.argmax(model_out) == 2:
        Submit.append([name,2])
    elif np.argmax(model_out) == 3:
        Submit.append([name,3])
    elif np.argmax(model_out) == 4:
        Submit.append([name,4])
    elif np.argmax(model_out) == 5:
        Submit.append([name,5])                                 
print(Submit)
Submited=pd.DataFrame(Submit,columns=['Image','Label'])
Submited.to_csv("/content/submit.csv", index = False) 

[['23004.jpg', 3], ['23215.jpg', 5], ['20998.jpg', 2], ['20627.jpg', 3], ['23885.jpg', 4], ['23291.jpg', 3], ['22374.jpg', 4], ['20683.jpg', 0], ['22636.jpg', 3], ['22623.jpg', 4], ['20647.jpg', 4], ['21277.jpg', 4], ['22450.jpg', 5], ['24115.jpg', 2], ['21967.jpg', 2], ['20283.jpg', 4], ['23597.jpg', 1], ['20751.jpg', 2], ['22311.jpg', 5], ['24213.jpg', 2], ['22346.jpg', 3], ['22400.jpg', 1], ['22490.jpg', 1], ['21936.jpg', 4], ['24175.jpg', 5], ['20334.jpg', 3], ['23024.jpg', 3], ['22149.jpg', 1], ['21017.jpg', 0], ['22268.jpg', 1], ['24132.jpg', 4], ['24250.jpg', 5], ['23429.jpg', 3], ['21230.jpg', 2], ['546.jpg', 0], ['23932.jpg', 0], ['22097.jpg', 5], ['23419.jpg', 4], ['22724.jpg', 3], ['20807.jpg', 5], ['22064.jpg', 5], ['21287.jpg', 5], ['20261.jpg', 1], ['949.jpg', 0], ['21186.jpg', 2], ['22895.jpg', 3], ['23177.jpg', 2], ['23876.jpg', 1], ['20726.jpg', 2], ['24133.jpg', 2], ['22614.jpg', 3], ['21294.jpg', 0], ['20096.jpg', 0], ['22158.jpg', 3], ['20435.jpg', 3], ['24297.jpg',